In [25]:
import pandas as pd

In [33]:
df = pd.read_parquet('tiling_results.parquet')

In [34]:
df.shape

(10, 11)

In [35]:
len(df[df.tiling_compatible == True])

4

In [36]:
df

,collection_concept_id,concept_id,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
0,C1748046084-LPCLOUD,MYD11A1.A2009004.h22v10.061.2021134002022,None,HDF-EOS2,hdf,False,unsupported_format,None,None,None,None
1,C1748058432-LPCLOUD,MOD11A1.A2007016.h20v02.061.2021055152114,None,HDF-EOS2,jpg,False,unsupported_format,None,None,None,None
2,C2069247178-GES_DISC,NLDAS_MOS0125_H.2.0:NLDAS_MOS0125_H.A20160524....,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Rainf,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
3,C2524245159-LPCLOUD,MOD11B1.A2000138.h16v09.061.2020044210723,None,HDF-EOS2,hdf,False,unsupported_format,None,None,None,None
4,C2033151148-GES_DISC,NLDAS_FORA0125_H.2.0:NLDAS_FORA0125_H.A2018111...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Tair,"[time_bnds, Tair, Qair, PSurf, Wind_E, Wind_N,..."
5,C2980666614-LAADS,LAADS:8396839114,xarray,HDF5,h5,False,cant_extract_variables,Could not extract data variables for granule L...,None,None,[]
6,C2069247359-GES_DISC,NLDAS_VIC0125_H.2.0:NLDAS_VIC0125_H.A20091025....,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Rainf,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
7,C2033167496-GES_DISC,NLDAS_FORB0125_H.2.0:NLDAS_FORB0125_H.A2017013...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Rainf,"[time_bnds, SWdown, Rainf, CRainf, ACond, Tair..."
8,C2343116525-LPCLOUD,MCD43A2.A2006324.h16v07.061.2020277015634,None,HDF-EOS2,hdf,False,unsupported_format,None,None,None,None
9,C2670138092-NSIDC_CPRD,ATL06_20181225075946_13390110_006_02.h5,xarray,None,h5,False,cant_extract_variables,Could not extract data variables for granule A...,None,None,[]
